## Export tidal corrections for the 10-minute North Sea tide gauge time vector

**<font color='red'>The clean version of this notebook was moved to the repository "P1-data-combination" on 29.11.2022.</font>**

with the idea to use it also for other datasets

Contains code parts from 4_tidal_prediction_aviso-fes

In [1]:
import pandas as pd
import pyfes
import numpy as np

Get the vector of dates from the tide gauge observation (10-minute vector from rijkswaterstaat North Sea tide gauge). Not every date has an observation, that's why the "with-999999999999999999"-version is loaded, but the ssh values are not of interest here.

In [52]:
datapath = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/6_tide_gauge/\
2_Ters-North_Sea_rijkswaterstaat/'
filename = 'TG_rijkswaterstaat_with-999999999_1992_01_01-2021_12_31.csv'
data = pd.read_csv(datapath + filename, index_col='datetime[utc]', \
                  parse_dates=['datetime[utc]'])

date = data.index.values.astype('datetime64[us]')
# coordinates of the North-Sea-Tide-Gauge
lat = np.full(len(date), 53.420034)
lon = np.full(len(date), 5.351231)

Checks for duplicates, NaNs and gaps in the date vector:

In [55]:
print("Duplicates: ", data.index.duplicated().sum())
print("NaNs: ", data.isnull().sum())

diffs = np.diff(date)
idx = np.where(diffs != diffs[0])
len(idx[0])

Duplicates:  0
NaNs:  ssh[cm]    0
dtype: int64


0

In [56]:
# datapaths to the tide models
path_eot = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/2_EOT20/'
path_fes = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/1_FES2014/'

# datapaths to the configuration files
path_eot_ocean_tide = path_eot + 'ocean_tide_wo-SA-SSA.ini'
path_eot_load_tide = path_eot + 'load_tide_wo-SA-SSA.ini'
path_fes_ocean_tide = path_fes + 'ocean_tide_extrapolated_red-to-eot_wo-SA-SSA.ini'
path_fes_load_tide = path_fes + 'load_tide_red-to-eot_wo-SA-SSA.ini'

In [57]:
ocean_tide_eot_handler = pyfes.Handler('ocean', 'io', path_eot_ocean_tide)
ocean_tide_eot = ocean_tide_eot_handler.calculate(lon, lat, date)

load_tide_eot_handler = pyfes.Handler('radial', 'io', path_eot_load_tide)
load_tide_eot = load_tide_eot_handler.calculate(lon, lat, date)

In [58]:
ocean_tide_fes_handler = pyfes.Handler('ocean', 'io', path_fes_ocean_tide)
ocean_tide_fes = ocean_tide_fes_handler.calculate(lon, lat, date)

load_tide_fes_handler = pyfes.Handler('radial', 'io', path_fes_load_tide)
load_tide_fes = load_tide_fes_handler.calculate(lon, lat, date)

In [62]:
# total correction (diurnal and semidiurnal + long period wave constituents)
corr_eot = ocean_tide_eot[0] + ocean_tide_eot[1] + load_tide_eot[0] + load_tide_eot[1]
corr_fes = ocean_tide_fes[0] + ocean_tide_fes[1] + load_tide_fes[0] + load_tide_fes[1]

In [63]:
# put date and both corrections in one dataframe
corr_df = pd.DataFrame(columns=['date', 'corr_eot[cm]', 'corr_fes[cm]'])
corr_df = corr_df.assign(date=data.index)
corr_df = corr_df.set_index(['date'])
corr_df['corr_eot[cm]'] = corr_eot
corr_df['corr_fes[cm]'] = corr_fes

# monthly averages
corr_df_monav = corr_df.groupby(pd.Grouper(freq='M')).mean()

In [64]:
savepath = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/'
corr_df.to_csv(savepath + "tidal_correction_10minutes.csv")
corr_df_monav.to_csv(savepath + "tidal_correction_monthly.csv")